# Largrid

In [ ]:
using BenchmarkTools
using LinearAlgebra
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
Cells = Array{Array{Int,1},1}

### grid(sequence::Array{Number,1})::Lar.LAR
Genera un modello LAR 1D. La funzione ritorna due elementi V ed EV che reppresentano rispettivamente la Geometria e la Topologia del modello. 

La funzione prende una sequenza di valori in ingresso e la trasforma in un array con il metodo `collect`.
Attraverso un ciclo for, vengono gradualmente popolati i due array `points` e `hulls` (solo quando i valori di `sequence` sono maggiori di 0), che vengono poi convertiti in Lar.

In [ ]:
function grid(sequence...)
	sequence = collect(sequence)
	cursor,points,hulls= (0,[[0.]],[])
	for value in sequence
		points = append!(points, [[cursor + abs(value)]])
		if value>=0
			append!(hulls,[[length(points)-1,length(points)]])
		end
	  cursor += abs(value)
	end
	V = convert(Lar.Points, [p[1] for p in points]')
	EV = convert(Lar.Cells,hulls)
	return V,EV
end

In [ ]:
@btime grid(1,-1,1,-1,1,-1,1,-1,1,-1)

### qn(n::Int)(sequence::Array{T,1})::Lar.LAR
Versione alternativa della funzione `grid` con un parametro di ripetizione `n`. La funzione nidificata `qn0` crea un modello Lar a partire dallo stesso input ricevuto da grid ma ripetendolo n volte.

In [ ]:
function qn(n::Int)
	function qn0(sequence::Array{T,1})::Lar.LAR  where T <: Real
		sequence = collect(sequence)
		return Lar.grid(repeat(sequence,outer=n)...)
	end
	return qn0
end

In [ ]:
@btime qn(3)([1.5,-2,0.5])

### grid_0(n::Int)::Array{Int64,2}
Genera un *complesso cellulare 0D* (punti) concatenando 0-celle ugualmente distanziate a intervalli unitari.

La funzione `hcat` si occupa di concatenare array per colonna, riceve in input un array di array contenenti un solo elemento rappresentante un punto.

`grid_0` ritorna questo 0-complesso in forma `Array{Int64,2}.

In [ ]:
function grid_0(n::Int)::Array{Int64,2}
    return hcat([[i] for i in range(0, length=n+1)]...)
end

In [ ]:
@btime grid_0(5)

### grid_1(n::Int)::Array{Int64,2}
Funzione gemella di `grid_0` genera un *complesso cellulare 1D* (segmenti) concatenando 1-celle ugualmente distanziate a intervalli unitari.

Facendo uso della funzione `hcat` concatena array di lunghezza 2, ognuno rappresentante un segmento e le sue due estremità.

`grid_1` ritorna questo 1-complesso in forma `Array{Int64,2}`.

In [ ]:
function grid_1(n)
    return hcat([[i,i+1] for i in range(0, length=n)]...)
end

In [ ]:
@btime grid_1(5)

### larGrid(n::Int)(d::Int)::Array{Int64,2}
Genera un *complesso cellulare 0D* oppure *un complesso cellulare 1D* a seconda del parametro `d` per cui sono accettati valori dal set ``{0,1}``.

`larGrid` fa uso delle funzioni `grid_0` e `grid_1` per generare i due tipi di complessi.

In [ ]:
function larGrid(n::Int)
    function larGrid1(d::Int)::Array{Int64,2}
        if d==0
         return grid_0(n)
        elseif d==1
         return grid_1(n)
        end
    end
    return larGrid1
end

In [ ]:
@btime larGrid(5)(1)

### cart(args::Array{Array{Any,1},1})::Array{Tuple,1}
Funzione che esegue il prodotto cartesiano delle collezioni inserite nell' array in input. Restituisce un `Array` di `Tuple`. Il numero di 
 unary `Array` argument. Return an `Array` of `Tuple`. The number `Tuple` restituito equivale al prodotto delle dimensione delle collezioni date in input.
 
Il metodo `product` ritorna un iteratore del prodotto di vari iteratori. Restituisce tutte le combinazioni possibili degli elementi delle collezioni in input sottoforma di `Tuple`.
`collect` trasforma l'output di `product` in un array i cui elementi vengono concatenati per riga dal metodo `vcat`.


In [ ]:
function cart(args)::Array{Tuple,1}
    return sort(vcat(collect(Iterators.product(args...))...))
 end

In [ ]:
@btime cart([[1,2],["a"],[3,4]])

### larVertProd(vertLists::Array{Points,1})::Points
Genera le coordinate intere (0-celle) di una *griglia multidimensionale*
Viene effettuato il prodotto cartesiano dei vertici contenuti in `vertLists` con la funzione `cart`. Le tuple restituite sono poi trasformate in formato Array per ottenere `coords`, una collezione di array bidimensionali. L'output è poi modificato nella forma dalle funzioni `hcat` e `sortslices`.

Sono messe a disposizione due versione della funzione `larVertProd`, che accettano rispettivamente `Array{Array{Int64,2},1}` e `Array{Float64,2},1}` come argomento

In [ ]:
function larVertProd(vertLists::Array{Array{Int64,2},1})::Array{Int64,2}
    coords = [[x[1] for x in v] for v in Lar.cart(vertLists)]
    return sortslices(hcat(coords...), dims=2)
 end
 function larVertProd(vertLists::Array{Array{Float64,2},1})::Array{Float64,2}
    coords = [[x[1] for x in v] for v in Lar.cart(vertLists)]
    return sortslices(hcat(coords...), dims=2)
 end

In [ ]:
@btime larVertProd([larGrid(2)(0), larGrid(2)(0)])

### index2addr(shape::Array{Int64,1})(multiIndex)::Int
Funzione che trasforma un *Multi-index in un indirizzo*. Multi-index è un generalizzazione del concetto di trasformazione di un indice intero in un *insieme ordinato di tuple di indici*. 
La funzione `index2addr` trasforma una lista di `shape` per un *array multidimensionale* in una funzione che, quando applicata ad un *array multi-index*, ad esempio una lista di interi di `Tuple` senza limiti di `shape`, ritorna gli *indirizzi interi* delle componenti dell'array corrispondenti all'interno della *memoria lineare* dell'array multidimensionale.


In [ ]:
function index2addr( shape::Array{Int64,2} )
    n = length(shape)
    theShape = append!(shape[2:end],1)
    weights = [prod(theShape[k:end]) for k in range(1, length=n)]

    function index2addr0( multiIndex::Array{Int,1} )::Int
        return dot(collect(multiIndex), weights) + 1
    end

    return index2addr0
end

In [ ]:
@btime index2addr([2,7])([1,4])

### larCellProd(cellLists::Array{Cells,1})::Cells
Genera una *griglia cellulare* dal *prodotto Cartesiano* di complessi 0/1-dimensionali.
Il complesso di output è generato dal prodotto di un qualsiasi numero di complessi cellulari 0- oppure 1-dimensionali. Il prodotto di ``d`` complessi 1-dimensionali generano un *solido ``d``-cellulare*, mentre il prodotto di ``n`` complesso 0-dimensionali e ``n-d`` complessi 1-dimensionali (``d < n``) generano un *Non-solido ``(n-d)``-cellulare*, inserito nel spazio ``n``-dimensionale.

In [ ]:
function larCellProd(cellLists::Array{Cells,1})::Cells
    shapes = [length(item) for item in cellLists]
    subscripts = cart([collect(range(0, length=shape)) for shape in shapes])
    indices = [collect(tuple) .+ 1 for tuple in subscripts]
 
    jointCells = [cart([cells[k] for (k,cells) in zip(index,cellLists)])
                    for index in indices]
    convertIt = index2addr([ (length(cellLists[k][1]) > 1) ? shape .+ 1 : shape
       for (k,shape) in enumerate(shapes) ])
    [vcat(map(convertIt, map(collect,jointCells[j]))...) for j in 1:length(jointCells)]
 end

In [ ]:
c1 = [[0,1],[1,2],[2,3]]
c0 = [[0],[1],[2]]
@btime larCellProd([c1, c1, c0])

### filterByOrder( n::Int )Array{Array{Array{Int8,1},1},1}
Filtra l'array di codici binari di n bit in base alla somma dei loro valori interi.

La variabile `terms` è una collezione di tutte le possibili combinazioni (array) di valori `0,1` con n (valore in input) a disposizione.

Questa collezione è poi restituita in forma ordinata in base alla somma dei valori degli array contenenti le combinazioni.

Prima riga: somma = 1,
Seconda riga: somma =2,
    ...
N-esima riga = somma = n

In [ ]:
function filterByOrder(n::Int)Array{Array{Array{Int8,1},1},1}
    terms = [[parse(Int8,bit) for bit in collect(term)] for term in Lar.binaryRange(n)]
    return [[term for term in terms if sum(term) == k] for k in 0:n]
 end

In [ ]:
@btime filterByOrder(2)

### larGridSkeleton( shape::Array{Int,1} )( d::Int )::Cells
Genera il `d`-scheletro di una griglia cuboidale con una data `shape`.

Ricordiamo che  il `d`-scheletro di un `p`-complesso cellulare (d<=p) è l'insieme delle `d`-celle che compongono il `p`-complesso. È quindi un suo sottocomplesso.

In [ ]:
function larGridSkeleton(shape)
    n = length(shape)
    function larGridSkeleton0( d::Int )::Cells

    	@assert d<=n

        components = filterByOrder(n)[d .+ 1]
        apply(fun,a) = fun(a)
		componentCellLists = [ [map(f,x)  for (f,x) in  zip( [larGrid(dim)
			for dim in shape], convert(Array{Int64,1},component) ) ]
				for component in components ]
        colList(arr) = [arr[:,k]  for k in 1:size(arr,2)]
        out = [ larCellProd(map(colList,cellLists)) for cellLists in componentCellLists ]
        return vcat(out...)
    end
    return larGridSkeleton0
end

In [ ]:
@btime larGridSkeleton([1,1,1])(3)

### larImageVerts(shape::Array{Int,1})::Array{Int64,2}
Linearizza la *grid of integer vertices*, data la `shape` di una *griglia cuboidale*.

In [ ]:
function larImageVerts( shape::Array{Int,1} )::Array{Int64,2}
    vertexDomain(n) = hcat([k for k in 0:n-1]...)
    vertLists = [vertexDomain(k+1) for k in shape]
    vertGrid = larVertProd(vertLists)
    return vertGrid
 end

In [ ]:
@btime larImageVerts([2,2,2])

### cuboidGrid( shape, filled=false )::Union( Cells, Array{Cells,1} )
Funzione generatrice multidimensionale. 
Genera o una griglia ``d``-dimensionale solida di ``d``-cuboidi nello spazio ``d``-dimensionale oppure l’array dello scheletro di dimensione ``p``, questo dipende dalla variabile Booleana `filled` in input. ``0``-cuboidi sono punti, ``1``-cuboidi sono segmenti, ``2``-cuboidi sono quadrati,  ``3``-cuboidi sono cubi, etc. I valori di `shape=[a,b,c]` determinano il numero ``a x b x c`` di ``d``-cellulari. Inoltre, `d = length(shape)`.

In [ ]:
function cuboidGrid( shape, filled=false )
    vertGrid = larImageVerts(shape)
    gridMap = larGridSkeleton(shape)
    if ! filled
       cells = gridMap(length(shape))
    else
       skeletonIds = 0:length(shape)
       cells = [ gridMap(id) for id in skeletonIds ]
    end
    return convert(Array{Float64,2},vertGrid), cells
 end

In [ ]:
@btime cuboidGrid([3,2,1],true)

### larModelProduct(modelOne, modelTwo)::Lar.LAR
La funzione prende in input due *Modelli LAR* e ne restituisce il loro *prodotto Cartesiano*.
Un *Modello Lar* è una coppia Geometria, Topologia dove la Geometria descrive come l'oggetto è immerso nello spazio mentre la Topologia è l'insieme di quelle relazioni come spigoli forniti attraverso i vertici, facce fornite attraverso i vertici e celle tridimensionali fornite attraverso i vertici.
Fondamentalmente un *Modello Lar* è una coppia di oggetti di coordinate e relazioni di adiacenza e incidenza tra celle.

In [ ]:
function larModelProduct( modelOne, modelTwo )
    (V, cells1) = modelOne
    (W, cells2) = modelTwo

    vertices = DataStructures.OrderedDict();
    k = 1
    for j in 1:size(V,2)
       v = V[:,j]
        for i in 1:size(W,2)
          w = W[:,i]
            id = [v;w]
            if haskey(vertices, id) == false
                vertices[id] = k
                k = k + 1
            end
        end
    end

    cells = []
    for c1 in cells1
        for c2 in cells2
            cell = []
            for vc in c1
                for wc in c2
                    push!(cell, vertices[[V[:,vc];W[:,wc]]] )
                end
            end
            push!(cells, cell)
        end
    end


    vertexmodel = []
    for v in keys(vertices)
        push!(vertexmodel, v)
    end
    verts = hcat(vertexmodel...)
    cells = [[v for v in cell] for cell in cells]
    return (verts, cells)
end

In [ ]:
function larModelProduct( twoModels )
    modelOne, modelTwo = twoModels
    larModelProduct(modelOne, modelTwo)
end

In [ ]:
geom,topol = [0. 1. 2.], [[1,2],[2,3]]
mod = (geom,topol)
@btime larModelProduct(mod, mod)

### INSR(f::Function)(seq::Array{Any,1})::Any
Trasforma una funzione binaria in una funzione n-aria.

In [ ]:
function INSR(f)
	function INSR0(seq)
		len = length(seq)
		res = seq[end]
		for i in range(len-2,step=-1,stop=0)
			res = f([seq[i+1], res])
		end
		return res
	end
	return INSR0
end

In [ ]:
mod1D = grid(repeat([.1,-.1],outer=5)...)
@btime INSR(larModelProduct)([mod1D,mod1D])